# 5.0 Hash Tables

## 5.1 Implement an LRU cache

### Problem Statement
Implement a least recently used (LRU) cache having the following interface:
* Constructor which takes maximum number of elements in the cache.
* `set(key, value)` sets the key equal to value.
  * If adding a new entry to the cache were to cause the cache to exceed the maximum capacity set in the constructor, then evict the least recently accessed entry.
  * If the key already exists in the cache, then mark this key as the most recently used.
  * This operation should complete in constant time.
* `get(key)` returns the value associated with key or None if no such key exists in the cache.
  * If the key exists in the cache, then mark this key as the most recently used.
  * This operation should complete in constant time.

In [1]:
import collections
import unittest


class LRUCache(object):
    """Least recently used cache with max capacity eviction policy."""
    
    def __init__(self, max_capacity=3):
        self.max_capacity = max_capacity
        self.entries = collections.OrderedDict()  # Cheating.
        
    def set(self, key, value):
        if key not in self.entries:
            if len(self.entries) == self.max_capacity:
                self.entries.popitem(last=False)
            self.entries[key] = value
        else:
            self.entries[key] = value
            self.entries.move_to_end(key)

    def get(self, key):
        value = self.entries.get(key)
        if value is not None:
            self.entries.move_to_end(key)
        return value


class LRUCacheTest(unittest.TestCase):

    def test_lru_cache(self):
        max_capacity = 3
        lru = LRUCache(max_capacity=max_capacity)
        
        # Fill to capacity.
        lru.set('k1', 'v1')
        lru.set('k2', 'v2')
        lru.set('k3', 'v3')
        
        lru.set('k4', 'v4')  # Evict k1.
        self.assertIsNone(lru.get('k1'))
        
        # Access k2 so that k3 becomes least recently used.
        self.assertEqual(lru.get('k2'), 'v2')
        lru.set('k5', 'v5')  # Evict k3.
        self.assertIsNone(lru.get('k3'))

        # Update k4 so that k2 becomes least recently used.
        lru.set('k4', 'v44')
        lru.set('k6', 'v6')  # Evict k2.
        self.assertIsNone(lru.get('k2'))


unittest.main(LRUCacheTest(), argv=[''], verbosity=2, exit=False)

test_lru_cache (__main__.LRUCacheTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


## 5.2 Cut brick wall

### Problem Statement
Given a wall of height $h$ built from bricks of irregular size.  Find the vertical line in the wall that requires the fewest number of bricks to be cut.  A vertical line in a location where 2 bricks meet requires no cut.

In [2]:
import collections
import unittest


def find_mininum_cut(wall):
    """Return vertical line through wall with fewest bricks to cut."""
    # Wall is represented by a matrix of brick lengths.
    # Sum of brick lengths in a row will be same (no holes in wall).

    # Record the count of bricks that end at vertical position.
    counts = collections.defaultdict(int)
    for row in wall:
        pos = 0
        for brick in row[:-1]:  # Ignore last brick along edge.
            pos += brick
            counts[pos] += 1

    # Find the vertical position having the most bricks which meet.
    maxpos, maxrows = 0, 0
    for pos, nrows in counts.items():
        if nrows > maxrows:
            maxpos, maxrows = pos, nrows

    numcuts = len(wall)-maxrows
    return (maxpos, numcuts)


class FindMinimumCutTest(unittest.TestCase):

    # Minimum cut through wall1 is 8 units from left and requires 2 cuts.
    wall1_expected = (8,2)
    wall1 = [
        [3,5,1,1],
        [2,3,3,2],
        [5,5],
        [4,4,2],
        [1,3,3,3],
        [1,1,6,1,1],
    ]

    def test_solution(self):
        case = collections.namedtuple('case', ['input','expected'])
        cases = [
            case(self.wall1, self.wall1_expected),
        ]
        for c in cases:
            rcv = find_mininum_cut(c.input)
            self.assertEqual(rcv, c.expected)


unittest.main(FindMinimumCutTest(), argv=[''], verbosity=2, exit=False)

test_solution (__main__.FindMinimumCutTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.009s

OK


## 5.3 Implement a sparse array

### Problem Statement